In [34]:
import serial
import time
import PySimpleGUI as sg

In [ ]:
rotors = [
    "EKMFLGDQVZNTOWYHXUSPAIBRCJ",
    "AJDKSIRUXBLHWTMCQGZNPYFVOE",
    "BDFHJLCPRTXVZNYEIWGAKMUSQO"
]
reflector = "IXUHFEZDAOMTKQJWNSRLCYPBVG"
turnovers = [ord('Q') - 65, ord('E') - 65, ord('V') - 65]
rotor_offsets = [0, 0, 0]

plugboard = [0] * 26 
for i in range(26):
    plugboard[i] = i



In [36]:
def char_to_index(c):
    return ord(c.upper()) - 65

def index_to_char(i):
    return chr((i%26) + 65) 

def rotor_r_to_l(c,r):
    index = (c + rotor_offsets[r])%26
    mapped = ord(rotors[r][index]) - 65
    res = (mapped - rotor_offsets[r])%26
    return res

def rotor_l_to_r(c, r):
    index = (c + rotor_offsets[r])%26
    inverse = rotors[r].index(index_to_char(index))
    res = (inverse - rotor_offsets[r])%26
    return res

def reflect(c):
    return ord(reflector[c]) - 65

def plug_swap(c):
    return plugboard[c]

def initialize_plugboard(pairs):
    global plugboard
    plugboard = [0]*26
    pairs = pairs.upper().split()
    for i in range(0, len(pairs), 2):
        if i+1 >= len(pairs):
            break
        a, b = char_to_index(pairs[i]), char_to_index(pairs[i+1])
        plugboard[a], plugboard[b] = b, a



In [37]:
def spin_rotors():
    rotor_offsets[0] = (rotor_offsets[0] + 1) % 26

    if (rotor_offsets[1] == turnovers[1] or rotor_offsets[0] == turnovers[0]):
        rotor_offsets[1] = (rotor_offsets[1] + 1) % 26
        if (rotor_offsets[1] == turnovers[1]):
            rotor_offsets[2] = (rotor_offsets[2] + 1) % 26

In [38]:
def encrypt_char(c):
    if not c.isalpha():
        return c
    c = char_to_index(c)
    c = plug_swap(c)
    spin_rotors()
    c = rotor_r_to_l(c, 0)
    c = rotor_r_to_l(c, 1)
    c = rotor_r_to_l(c, 2)
    c = reflect(c)
    c = rotor_l_to_r(c, 2)
    c = rotor_l_to_r(c, 1)
    c = rotor_l_to_r(c, 0)
    c = plug_swap(c)

    return index_to_char(c)



In [39]:
def encrypt_message(message, update_callback):
    encrypted = ""
    i = 0
    while i < len(message):
        ch = message[i]
        if ch == ' ':
            new_config = sg.popup_get_text("Space found. Update plugboard? Enter new pairs or leave empty to keep current:", default_text="")
            if new_config:
                initialize_plugboard(new_config)
            i += 1
            continue
        encrypted += encrypt_char(ch)
        update_callback(encrypted)
        i += 1
    return encrypted

def set_rotor_positions(left, middle, right):
    rotor_offsets[2] = char_to_index(left)
    rotor_offsets[1] = char_to_index(middle)
    rotor_offsets[0] = char_to_index(right)

In [40]:
layout = [
    [sg.Text("Enter Message to Encrypt:")],
    [sg.Input(key='-INPUT-', size=(40, 1))],
    [sg.Button("Encrypt", key='-ENCRYPT-')],
    [sg.Output(size=(60, 15), key='-OUTPUT-')]
]

window = sg.Window("Enigma Simulator", layout)

In [41]:
while True:
    event, values = window.read(timeout=100)

    if event == sg.WINDOW_CLOSED:
        break

    if event == '-ENCRYPT-':
        message = values['-INPUT-'].upper()
        encrypted = ""
        plug_pairs = "A A B B C C D D E E F F G G H H I I J J K K L L M M N N O O P P Q Q R R S S T T U U V V W W X X Y Y Z Z"  # Default pairs

        initialize_plugboard(plug_pairs)
        for ch in message:
            if ch == ' ':
                # Ask user for new plugboard config
                new_pairs = sg.popup_get_text(
                    "Space found. Update Steckerboard pairs (A B C D), or press OK to keep current:",
                    default_text=plug_pairs
                )
                if new_pairs:
                    plug_pairs = new_pairs
                initialize_plugboard(plug_pairs)
                encrypted += ' '
            else:
                encrypted_char = encrypt_char(ch)
                encrypted += encrypted_char

        print(f"\nOriginal:  {message}")
        print(f"Encrypted: {encrypted}")

window.close()